In [ ]:
# ==============================================================================
# CELL 1: INSTALL ALL DEPENDENCIES
# ==============================================================================
# This cell installs all necessary packages. The last line will automatically
# restart the Colab runtime, which is required for the new packages to be
# correctly loaded.
# ==============================================================================
import os

# Install system utilities
print("[INFO] Installing ffmpeg and git...")
!apt-get update -qq && apt-get install -y -qq ffmpeg git > /dev/null

# Install Python packages for downloading and processing
print("[INFO] Installing Python packages...")
# Upgrade torch to a version >= 2.6 to fix the security vulnerability
!pip install --upgrade -q torch torchvision torchaudio

# Install whisper-diarization dependencies
!pip install -q "ctranslate2>=4.5.0" "faster-whisper>=1.0.0"
!pip install -q "nemo-toolkit[asr]>=2.dev"
!pip install -q git+https://github.com/MahmoudAshraf97/demucs.git
!pip install -q git+https://github.com/oliverguhr/deepmultilingualpunctuation.git
!pip install -q git+https://github.com/MahmoudAshraf97/ctc-forced-aligner.git
!pip install -q yt-dlp

print("\\n[INFO] All dependencies installed.")
print("[IMPORTANT] Restarting runtime to apply changes...")

# Restart the runtime to load the new packages
os.kill(os.getpid(), 9)

[INFO] Installing ffmpeg and git...
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
[INFO] Installing Python packages...
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.
shell-init: error retrieving curre

In [ ]:
# ==============================================================================
# CELL 2: DOWNLOAD AUDIO, RUN DIARIZATION, AND SAVE RESULTS (Corrected)
# ==============================================================================
# This script performs the entire workflow:
# 1. Clones the whisper-diarization repository.
# 2. Downloads audio from a specified YouTube URL.
# 3. Sets the LD_LIBRARY_PATH to fix cuDNN version conflicts.
# 4. Runs the diarization and transcription process.
# 5. Saves the output (.srt, .txt) to a results folder and creates a ZIP file.
# ==============================================================================
import os
import sys
import shutil
import pathlib
import subprocess
import yt_dlp
import torch

# --- Configuration ---
YOUTUBE_URL   = "https://www.youtube.com/watch?v=LJiUDxj-2ZE"
REPO_URL      = "https://github.com/MahmoudAshraf97/whisper-diarization.git"
REPO_DIR      = pathlib.Path("/content/whisper-diarization")
AUDIO_DIR     = pathlib.Path("/content/audios")
RESULTS_DIR   = pathlib.Path("/content/resultados")
WHISPER_MODEL = "medium"
ENTRYPOINT    = "diarize.py"

# --- 1. Clone the repository ---
print(f"[INFO] Cloning repository from {REPO_URL}...")
if REPO_DIR.exists():
    shutil.rmtree(REPO_DIR)
subprocess.run(
    ["git", "clone", REPO_URL, str(REPO_DIR)],
    check=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)
print("[OK] Repository cloned.")

# --- 2. Download audio from YouTube ---
AUDIO_DIR.mkdir(parents=True, exist_ok=True)
ytdl_opts = {
    "format": "bestaudio/best",
    "outtmpl": str(AUDIO_DIR / "%(title)s.%(ext)s"),
    "postprocessors": [{
        "key": "FFmpegExtractAudio",
        "preferredcodec": "mp3",
        "preferredquality": "192",
    }],
    "quiet": True,
}

print(f"[INFO] Downloading audio from YouTube URL: {YOUTUBE_URL}...")
with yt_dlp.YoutubeDL(ytdl_opts) as ydl:
    info = ydl.extract_info(YOUTUBE_URL, download=True)
    downloaded_file = ydl.prepare_filename(info)
    audio_path = pathlib.Path(downloaded_file).with_suffix(".mp3")

if not audio_path.exists():
    raise FileNotFoundError(f"Failed to download or find the audio file at {audio_path}")
print(f"[OK] Audio downloaded successfully: {audio_path.name}")

# --- 3. Fix cuDNN version incompatibility ---
# Set LD_LIBRARY_PATH to prioritize PyTorch's bundled cuDNN library.
torch_lib_path = pathlib.Path(torch.__file__).parent / "lib"
ld_library_path = os.environ.get("LD_LIBRARY_PATH", "")
os.environ["LD_LIBRARY_PATH"] = f"{torch_lib_path}:{ld_library_path}"
print(f"[INFO] Set LD_LIBRARY_PATH to prioritize bundled cuDNN.")


# --- 4. Run the diarization pipeline ---
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
os.chdir(REPO_DIR)

print(f"[INFO] Starting diarization process for {audio_path.name}...")
cmd = [
    sys.executable, "-u", ENTRYPOINT,
    "--whisper-model", WHISPER_MODEL,
    "-a", str(audio_path)
]

print(f"[CMD] {' '.join(cmd)}")
logfile = RESULTS_DIR / "run_diarization.log"

# Run the command with the updated environment
with open(logfile, "w", encoding="utf-8") as lf:
    result = subprocess.run(
        cmd,
        stdout=lf,
        stderr=subprocess.STDOUT,
        text=True,
        encoding='utf-8',
        env=os.environ.copy() # Pass the modified environment
    )

# Print the log to the console for real-time feedback
with open(logfile, "r", encoding="utf-8") as lf:
    print(lf.read())

print(f"\n[INFO] Process finished with return code: {result.returncode}")
if result.returncode != 0:
    raise RuntimeError(f"Diarization failed. Check the log file: {logfile}")

# --- 5. Collect and package results ---
base_name = audio_path.stem
copied_files = []
for ext in [".srt", ".txt"]:
    src_file = REPO_DIR / f"{base_name}{ext}"
    if src_file.exists():
        dst_file = RESULTS_DIR / src_file.name
        shutil.copy2(src_file, dst_file)
        copied_files.append(dst_file.name)

if copied_files:
    print("\n[OK] Output files copied to results folder:")
    for f in copied_files:
        print(f"  - {f}")
else:
    print("\n[WARN] No output files (.srt, .txt) were generated.")

# Create a ZIP archive of the results
zip_out = RESULTS_DIR / f"{base_name}_results"
shutil.make_archive(str(zip_out), "zip", RESULTS_DIR)
print(f"[OK] All results have been packaged into: {zip_out}.zip")

[INFO] Cloning repository from https://github.com/MahmoudAshraf97/whisper-diarization.git...
[OK] Repository cloned.
[INFO] Downloading audio from YouTube URL: https://www.youtube.com/watch?v=LJiUDxj-2ZE...
[OK] Audio downloaded successfully: Así es VIVIR y ESCAPAR de una CÁRCEL en PERÚ.mp3
[INFO] Set LD_LIBRARY_PATH to prioritize bundled cuDNN.
[INFO] Starting diarization process for Así es VIVIR y ESCAPAR de una CÁRCEL en PERÚ.mp3...
[CMD] /usr/bin/python3 -u diarize.py --whisper-model medium -a /content/audios/Así es VIVIR y ESCAPAR de una CÁRCEL en PERÚ.mp3
2025-10-08 15:53:21.033105: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759938801.053480    9501 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759938801.059603    9501 cuda_blas

RuntimeError: Diarization failed. Check the log file: /content/resultados/run_diarization.log